In [1]:
!pip install flask_ngrok

In [2]:
!pip install -U xgboost

     |████████████████████████████████| 173.6 MB 10.0 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [3]:
!pip install pyngrok
!ngrok authtoken 26BsE9vqKr5gfvRuNAdiVxDg0xf_56FMXsvYW4TyZm7Mi3xJo

     |████████████████████████████████| 745 kB 11.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=9a3a282ccc0798ba8a76690d8ac9f1496029158402f46a60c59d1d59bf58c855
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%load_ext rpy2.ipython

In [6]:
%%R
install.packages("PSSMCOOL")
library(PSSMCOOL)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘gtools’, ‘infotheo’, ‘phonTools’, ‘dtt’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/gtools_3.9.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 240402 bytes (234 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [7]:
import pandas as pd

In [8]:
filenm=""

In [9]:
%%R -i filenm -o f1
f1<-function(filenm)
{
  d1<-NULL
  d1<-rbind(d1,dpc_pssm(filenm))
  write.csv(d1,file="/content/t1.csv")  
}

In [10]:
%%R -i filenm -o f2
f2<-function(filenm)
{
  d1<-NULL
  d1<-rbind(d1,DFMCA_PSSM(filenm))
  write.csv(d1,file="/content/t2.csv") 
}

In [11]:
%%R -i filenm -o f3
f3<-function(filenm)
{
  d1<-NULL
  d1<-rbind(d1,CS_PSe_PSSM(filenm, vec_name="total"))
  write.csv(d1,file="/content/t3.csv")  
}

In [12]:
%%R -i filenm -o f4
f4<-function(filenm)
{
  d1<-NULL
  d1<-rbind(d1,MBMGACPSSM(filenm))
  write.csv(d1,file="/content/t4.csv")  
}

In [13]:
from google.colab import files
uploaded = files.upload()

Saving templates.zip to templates.zip


In [14]:
import zipfile
from zipfile import ZipFile

file_name = '/content/templates.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Extracted Successfully')

Extracted Successfully


In [15]:
import flask
from flask import Flask, render_template, request, redirect, jsonify, url_for, session
import pandas as pd
import pickle
from flask_ngrok import run_with_ngrok
import warnings
from werkzeug.utils import secure_filename
from pip import main
warnings.filterwarnings('ignore')

app = Flask(__name__)
UPLOAD_FOLDER = '/content/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

run_with_ngrok(app)

df=pd.read_csv("/content/drive/MyDrive/SCOP_App/data/data.csv")
model1 = pickle.load(open("/content/drive/MyDrive/SCOP_App/model.dat","rb"))
model2=pickle.load(open("/content/drive/MyDrive/SCOP_App/model2.dat","rb"))
index=-1

def predict(pdb):
    for i in range(0,13035):
        if(pdb==df.iloc[i,0]):
            index=i
            print(index)
            tmp=df.iloc[index,1:-1]
            result=pd.DataFrame([tmp])
            return model1.predict(result)
    return -1

def PredictFromPssm(filenm):
    f1(filenm)
    f2(filenm)
    f3(filenm)
    f4(filenm)

    df1=pd.read_csv("t1.csv")
    df1= df1.drop(df1.columns[0], axis=1)

    df2=pd.read_csv("t2.csv")
    df2= df2.drop(df2.columns[0], axis=1)

    df3=pd.read_csv("t3.csv")
    df3= df3.drop(df3.columns[0], axis=1)

    df4=pd.read_csv("t4.csv")
    df4= df4.drop(df4.columns[0], axis=1)

    df5 = pd.concat([df1,df2,df3,df4], axis=1)

    result=model2.predict(df5)
    return result


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/pdb',methods=['POST'])
def pdb():
        pdbid = request.form.get('pdbid')
        result = predict(pdbid)
        if result==0:
          result='α'
        elif result==1:
          result='ß'
        elif result==2:
          result='α/ß'
        elif result==3:
          result='α+ß'
        else:
          result='Invalid'
        return render_template('pdb.html',pdbid=pdbid, result=result)

@app.route('/pssm',methods=['POST'])
def pssm():
        f =request.files['file']
        f.save(secure_filename(f.filename))
       
        filename = "/content/"+f.filename

        result=PredictFromPssm(filename)
        if result==0:
          result='α'
        elif result==1:
          result='ß'
        elif result==2:
          result='α/ß'
        elif result==3:
          result='α+ß'
        else:
          result='Invalid'

        return render_template('pssm.html', result=result)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0dc4-34-77-142-121.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Mar/2022 16:31:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2022 16:31:05] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Mar/2022 16:31:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2022 16:31:08] "GET /about HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2022 16:31:09] "GET /... HTTP/1.1" 404 -
127.0.0.1 - - [27/Mar/2022 16:31:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2022 16:31:31] "POST /pdb HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2022 16:31:32] "GET /pdb HTTP/1.1" 405 -
127.0.0.1 - - [27/Mar/2022 16:31:49] "GET /pssm HTTP/1.1" 405 -
127.0.0.1 - - [27/Mar/2022 16:31:50] "POST /pssm HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2022 16:31:55] "GET /about HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2022 16:31:56] "GET /... HTTP/1.1" 404 -


In [16]:
model2=pickle.load(open("/content/drive/MyDrive/SCOP_App/model2.dat","rb"))

In [17]:
def PredictFromPssm(filenm):
    f1(filenm)
    f2(filenm)
    f3(filenm)
    f4(filenm)

    df1=pd.read_csv("t1.csv")
    df1= df1.drop(df1.columns[0], axis=1)

    df2=pd.read_csv("t2.csv")
    df2= df2.drop(df2.columns[0], axis=1)

    df3=pd.read_csv("t3.csv")
    df3= df3.drop(df3.columns[0], axis=1)

    df4=pd.read_csv("t4.csv")
    df4= df4.drop(df4.columns[0], axis=1)

    df5 = pd.concat([df1,df2,df3,df4], axis=1)

    result=model2.predict(df5)
    return result

In [ ]:
result=PredictFromPssm("/content/test.pssm")
print(result)

[2]
